In [1]:
import pandas as pd
import numpy as np
import requests
import urllib.request
from bs4 import BeautifulSoup as bs
import copy
import re
import datetime as dt
from time import time
from time import sleep
from warnings import warn

import pyodbc
from sqlalchemy import create_engine

# Connecting to SQL Server

In [2]:
SERVER = 'tx-dot-dev-sqlsvr.database.windows.net'
DATABASE = 'TX-DOT-DEV-DB'
USERNAME = 'sysadmin'
PWD = 'N0ru_Mu5k09a9'
TABLE = "Public_Project_Data"

driver= '{ODBC Driver 17 for SQL Server}'  

In [3]:
connection_string = 'DRIVER=' + driver + \
                    ';SERVER=' + SERVER + \
                    ';PORT=1433' + \
                    ';DATABASE=' + DATABASE + \
                    ';UID=' + USERNAME + \
                    ';PWD=' + PWD

params = urllib.parse.quote_plus(connection_string)
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
sql = 'SELECT MAX([date_accessed]) FROM [stg].[Analysis_Ready_Data]'
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()
df = pd.read_sql(sql, conn)

### Querying the Latest Date the Data Was Updated

In [4]:
def get_last_date():
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    cursor.execute('SELECT MAX([date_accessed]) FROM [stg].[Analysis_Ready_Data]')
    
    for row in cursor:
        return row[0]

In [5]:
latest_date = get_last_date()
latest_date

datetime.date(2019, 6, 21)

### Values that will be used to create the url's to scrape data starting from latest Update Date

In [6]:
#latest month and year
month_init = int(latest_date.month)
year_init = int(latest_date.year)

if month_init == 12:
    month_init = 1
    year_init += 1
else:
    month_init +=1

project_type = ['sc','ll','sm'] #state construction, local letting, state maintenance
year = [str(i) for i in range(year_init,dt.datetime.today().year+1)]
month = [str(i) for i in range(1,13)] #all possible months

In [7]:
base_url = 'http://www.dot.state.tx.us/insdtdot/orgchart/cmd/cserve/bidtab/'
headers = {"Accept-Language": "en-US, en;q=0.5"}

tot_data = []
local_table = []

# Scrapes All Data From Webpage

- Loops through all possible urls from latest update date
- Checks that the Tab Page exists
- If so adds all the data in the table into tot_data

In [8]:
def web_scraper():
    tot_data[:] = []
    local_table[:] = []
    first_month = True
    month_cnt = 1
    num_req = 0 #number of requests perfomed by html scraper
    start_time = time() #start time of running the scraper
    for proj_type in project_type:
        for yr in year:
            while month_cnt < len(month):
                if first_month:
                    month_cnt = month_init
                    first_month = False
                response = (requests.get(base_url+proj_type+yr+'0'+month[month_cnt]+'.htm',headers = headers) if(len(month[month_cnt])<2) else requests.get(base_url+proj_type+yr+month[month_cnt]+'.htm',headers = headers))
                if response.status_code == 200:
                    html_parser = bs(response.text , 'lxml',parse_only = ss('table')
                    info_table = html_parser.findAll('table')
                    if(len(html_parser.findAll('table')) != 0):
                        info_table = html_parser.findAll('table')[1]
                        it_rows = info_table.findAll('tr')
                        for i in range(1,len(it_rows)):
                            web_id = it_rows[i].findAll('td')[0].get_text().strip().replace('/','') + it_rows[i].findAll('td')[1].get_text().strip()
                            temp_scrape = bs(requests.get(base_url+web_id + '.htm').text,'lxml',parse_only = ss(['table','title']))
                            if(temp_scrape.find('title').get_text() == 'Page Not Found'):
                                continue
                            else:
                                num_req += 1
                                print(str(num_req) + ": " + base_url+web_id + '.htm')
                                tot_data.append((temp_scrape.findAll('table')[0:2]))
                month_cnt += 1

SyntaxError: invalid syntax (<ipython-input-8-8d5ebafb6394>, line 17)

# Conversion to DataFrame

- Goes through all the scraped data and adds it to local_table under the correct columns
- Goes through both main url data and data from the Tabs url

In [ ]:
def create_df()    
    count = 0
    for cols in range(0,len(tot_data[0][count].findAll('td')[1:])):
        count += 1
        if cols%2 != 0:
            local_table.append((tot_data[0][0].findAll('td')[cols].get_text().strip(),[]))
    local_table.append(('Estimate: ',[]))
    local_table.append(('Bid: ',[]))
    local_table.append(('Over / Under %' ,[]))
    local_table.append(('Bidder: ',[]))
    local_table.append(('Winning Bidder: ',[]))
    for i in range(0,len(tot_data)):
        count = 1
        for j in range(1,len(tot_data[i][0].findAll('td'))):
            if j%2 == 0:
                local_table[count-1][1].append(tot_data[i][0].findAll('td')[j].get_text())
                count+=1
            if count >= len(local_table) -4:
                count = 1
        tot_bidders = 0
        txt = []
        for j in range(0,len(tot_data[i][1].findAll('td'))):
            if count == 1:
                local_table[15][1].append(tot_data[i][1].findAll('td')[1].get_text().strip())
                count+=1
            temp = tot_data[i][1].findAll('td')[j].get_text().strip()
            tot_bidders += temp.count('Bidder')
            txt.append(temp)
        bid_count = 1
        txt = txt[4:]
        winning_bidder = txt[3]
        for k in range(0,tot_bidders):
            if(k > 0):
                for l in range(0,len(local_table)-4):
                    local_table[l][1].append(local_table[l][1][len(local_table[l][1])-1])
            local_table[16][1].append(txt[bid_count])
            local_table[17][1].append(txt[bid_count+1])
            local_table[18][1].append(txt[bid_count+2])
            local_table[19][1].append(winning_bidder)
            bid_count += 4
    Dict={title:column for (title,column) in local_table}
    df = pd.DataFrame(Dict)
    # Pushes Staging Data to Database
    df.to_sql(TABLE, engine, if_exists='append')
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    #Calls Stored Procedure to Store Data Properly
    cursor.execute("{call dbo.Stage_Raw_Data}")
    cursor.commit()
    

In [ ]:
web_scraper()

In [ ]:
create_df()